In [ ]:
#Auto-reload modules (used to develop functions outside this notebook)
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import numpy as np
import pandas as pd
import labrotation.file_handling as fh
import datadoc_util as ddutil
from labrotation import two_photon_session as tps
import h5py
from datetime import datetime as dt
from datetime import timedelta

In [ ]:
SERVER_SYMBOL = "R"

In [ ]:
env_dict = dict()
if not os.path.exists("./.env"):
    print(".env does not exist")
else:
    with open("./.env", "r") as f:
        for line in f.readlines():
            l = line.rstrip().split("=")
            env_dict[l[0]] = l[1]
print(env_dict.keys())

In [ ]:
output_dir = fh.open_dir("Choose export directory for results!")

In [ ]:
ddoc = ddutil.DataDocumentation(env_dict["DATA_DOCU_FOLDER"])

In [ ]:
ddoc.loadDataDoc()

In [ ]:
def get_datetime_for_fname():
    now = dt.now()
    return f"{now.year:04d}{now.month:02d}{now.day:02d}-{now.hour:02d}{now.minute:02d}{now.second:02d}"

In [ ]:
matlab_2p_folder = env_dict["MATLAB_2P_FOLDER"]

# Define recordings to be used

In [ ]:
df_groupings = ddoc.getRecordingsWithExperimentType(["chr2_ctl", "chr2_sz", "chr2_szsd", "chr2_sd", "chr2_lfpsz_sd"])

In [ ]:
# do the filtering by mouse
df_groupings = df_groupings[df_groupings["mouse_id"].isin(["T370", "T413", "T430", "T452", "OPI-2239"])]

In [ ]:
df_groupings.head()

# Consistency check
If no output, then everything looks fine

In [ ]:
for i_grouping, grouping in df_groupings.iterrows():
    nd2_fpath = SERVER_SYMBOL + os.path.join(grouping["folder"], grouping["nd2"])[1:]  # replace server symbol
    lv_fpath = SERVER_SYMBOL + os.path.join(grouping["folder"], grouping["labview"])[1:] 
    lvtime_fpath = SERVER_SYMBOL + os.path.join(grouping["folder"], os.path.splitext(grouping["labview"])[0]+"time.txt")[1:] 
    nikmeta_fpath = SERVER_SYMBOL + os.path.join(grouping["folder"], grouping["nikon_meta"])[1:] 
    
    
    if type(grouping["lfp"]) is float:  # NaN has type float, otherwise should be string
        lfp_fpath = None
    else:
        lfp_fpath =  SERVER_SYMBOL + os.path.join(grouping["folder"], grouping["lfp"])[1:] 
        if not os.path.exists(lfp_fpath):
            print(lfp_fpath)
    if not os.path.exists(nd2_fpath):
        print(nd2_fpath)
    if not os.path.exists(lv_fpath):
        print(lv_fpath)
    if not os.path.exists(lvtime_fpath):
        print(lvtime_fpath)
    if not os.path.exists(nikmeta_fpath):
        print(nikmeta_fpath)

In [ ]:
append_to_file = True
if append_to_file:
    existing_uuids = []
    original_fpath = fh.open_file("Choose h5 file to append to!")
    with h5py.File(original_fpath, "r") as hf:
        for uuid in hf.keys():
            existing_uuids.append(uuid)

In [ ]:
sessions_dict = dict()
has_lfp_dict = dict()
for i_grouping, grouping in df_groupings.iterrows():
    uuid = grouping.uuid
    if append_to_file:
        if uuid in existing_uuids:
            print(f"skipping {uuid}...")
            continue
    
    nd2_fpath = SERVER_SYMBOL + os.path.join(grouping["folder"], grouping["nd2"])[1:]  # replace server symbol
    lv_fpath = SERVER_SYMBOL + os.path.join(grouping["folder"], grouping["labview"])[1:] 
    lvtime_fpath = SERVER_SYMBOL + os.path.join(grouping["folder"], os.path.splitext(grouping["labview"])[0]+"time.txt")[1:] 
    nikmeta_fpath = SERVER_SYMBOL + os.path.join(grouping["folder"], grouping["nikon_meta"])[1:] 
    
    has_lfp = True
   
    
    if type(grouping["lfp"]) is float:  # NaN has type float, otherwise should be string
        lfp_fpath = None
        has_lfp = False
    else:
        lfp_fpath =  SERVER_SYMBOL + os.path.join(grouping["folder"], grouping["lfp"])[1:] 
    
    session = tps.TwoPhotonSession.init_and_process(nd2_fpath, nikmeta_fpath, lv_fpath, lvtime_fpath, lfp_fpath, matlab_2p_folder)
    
    sessions_dict[uuid] = session
    has_lfp_dict[uuid] = has_lfp

In [ ]:
if not append_to_file:
    output_assembled_fpath = os.path.join(output_dir, f"assembled_traces_{get_datetime_for_fname()}_ChR2.h5")
    print(f"Saving traces to {output_assembled_fpath}")

In [ ]:
def create_dt(t):
    t1 = t[1:]
    t0 = t[:-1]
    dt = np.zeros(len(t))
    dt[1:] = t1 - t0
    dt[0] = dt[1]  # assume same step size to avoid 0
    return dt
def create_totdist_abs(speed, dt):
    totdist_abs = np.zeros(len(speed))
    totdist_abs[0] = speed[0]*dt[0]
    for i in range(1, len(totdist_abs)):
        totdist_abs[i] = totdist_abs[i-1] + abs(speed[i]*dt[i])
    return totdist_abs

In [ ]:
if append_to_file:
    export_fpath = original_fpath
    access_type = "r+"
else:
    access_type = "w-"
    export_fpath = output_assembled_fpath
with h5py.File(export_fpath, access_type) as hf:
    for event_uuid in sessions_dict.keys():
        
        if append_to_file:
            if event_uuid in existing_uuids:
                continue
        
        session = sessions_dict[event_uuid]
        exp_type = ddoc.getExperimentTypeForUuid(event_uuid) 
        
        segments = ddoc.getSegmentsForUUID(event_uuid).sort_values(by="frame_begin")
        if len(segments[segments["interval_type"] == "stimulation"]) > 0:
            stim_segment = segments[segments["interval_type"] == "stimulation"].iloc[0]
        else:
            print(event_uuid)
            raise Exception()
        n_frames = len(sessions_dict[event_uuid].mean_fluo)
        
        # bl frames: beginning until stimulation
        # am frames: from first SD wave frame until end (to be consistent with TMEV am definition)
        #            if no SD: then from end of stimulation
        n_bl_frames = stim_segment["frame_begin"] - 1  # frames 1 to x = x frames. stim_segment["frame_begin"] is x+1 
        n_stim_frames = stim_segment["frame_end"] - stim_segment["frame_begin"] + 1 # both end points inclusive -> +1 
        
        if "sd_wave" in segments.interval_type.unique():
            # first frame of SD wave until end
            n_am_frames = n_frames - segments[segments["interval_type"] == "sd_wave"].iloc[0].frame_begin + 1
        else:
            # first frame AFTER end of stim until end
            n_am_frames = n_frames - segments[segments["interval_type"] == "stimulation"].iloc[0].frame_end
        event_uuid_grp = hf.create_group(event_uuid)
        #df_attributes = df_events[df_events["event_uuid"] == event_uuid]
        
        mouse_id = ddoc.getMouseIdForUuid(event_uuid)
        
        event_uuid_grp.attrs["session_uuids"] = [event_uuid]  # only one uuid, as ChR2 protocol is always single recording
        event_uuid_grp.attrs["has_lfp"] = [has_lfp_dict[event_uuid]]
        event_uuid_grp.attrs["window_type"] =  ddoc.getMouseWinInjInfo(mouse_id).window_type.iloc[0]
        event_uuid_grp.attrs["n_frames"] = n_frames
        event_uuid_grp.attrs["mouse_id"] = mouse_id
        event_uuid_grp.attrs["exp_type"] = exp_type
        

        event_uuid_grp.attrs["n_bl_frames"] = n_bl_frames
        event_uuid_grp.attrs["n_am_frames"] = n_am_frames
        
        
        lv_dist = session.belt_scn_dict['distance']
        lv_speed = session.belt_scn_dict['speed']
        lv_running = session.belt_scn_dict['running']
        lv_totdist = session.belt_scn_dict['totdist']
        lv_rounds = session.belt_scn_dict['rounds']
        lv_t_s = session.belt_scn_dict['tsscn']/1000.
        mean_fluo = session.mean_fluo
        
        lv_dt = create_dt(lv_t_s)
        lv_totdist_abs = create_totdist_abs(lv_speed, lv_dt)
        


        # get lfp data
        # lfp already matched to labview. lfp and movement channels t values should be same, but save them to be sure

        if has_lfp_dict[event_uuid]:
            lfp_t, lfp_y = session.lfp_lfp()
            lfp_mov_t, lfp_mov_y = session.lfp_movement()
        else:
            lfp_t = lv_t_s.copy()
            lfp_mov_t = lv_t_s.copy()

            lfp_y = np.zeros(len(lfp_t))
            lfp_mov_y = np.zeros(len(lfp_t))
                
        event_uuid_grp.attrs["n_lfp_steps"] = len(lfp_t)
        event_uuid_grp.attrs["n_lfp_mov_steps"] = len(lfp_mov_t)
        
        event_uuid_grp.create_dataset("lfp_mov_t", data=lfp_mov_t)
        event_uuid_grp.create_dataset("lfp_mov_y", data=lfp_mov_y)
        event_uuid_grp.create_dataset("lfp_t", data=lfp_t)
        event_uuid_grp.create_dataset("lfp_y", data=lfp_y)
        event_uuid_grp.create_dataset("lv_dist", data=lv_dist)
        event_uuid_grp.create_dataset("lv_dt", data=lv_dt)
        event_uuid_grp.create_dataset("lv_rounds", data=lv_rounds)
        event_uuid_grp.create_dataset("lv_running", data=lv_running)
        event_uuid_grp.create_dataset("lv_speed", data=lv_speed)
        event_uuid_grp.create_dataset("lv_t_s", data=lv_t_s)
        event_uuid_grp.create_dataset("lv_totdist", data=lv_totdist)
        event_uuid_grp.create_dataset("lv_totdist_abs", data=lv_totdist_abs)
        event_uuid_grp.create_dataset("mean_fluo", data=mean_fluo)
        
        
        # replicate joint_session_metadata_dict from tmev assembled traces
        if "sd_wave" in segments.interval_type.unique():
            break_points = np.array([0, n_bl_frames, n_bl_frames + n_stim_frames, segments[segments["interval_type"] == "sd_wave"].iloc[0].frame_begin + 1])
        else:
            break_points = np.array([0, n_bl_frames, n_bl_frames + n_stim_frames, segments[segments["interval_type"] == "stimulation"].iloc[0].frame_end + 1])
        
        event_uuid_grp.attrs["break_points"] = break_points
        event_uuid_grp.attrs["break_points_lfp"] = np.searchsorted(lfp_t, lv_t_s[break_points])
        
        segment_type_break_points = np.array([row["frame_begin"] - 1 for i_row, row in ddoc.getSegmentsForUUID(event_uuid).sort_values(by="frame_begin").iterrows()])
        event_uuid_grp.attrs["segment_type_break_points"] = segment_type_break_points
        event_uuid_grp.attrs["segment_type_break_points_lfp"] = np.searchsorted(lfp_t, lv_t_s[segment_type_break_points])
        event_uuid_grp.attrs["recording_break_points"] =  np.array([0])
        event_uuid_grp.attrs["recording_break_points_lfp"] = np.array([0]) 
        
        event_uuid_grp.attrs["recording_break_points_lfp"] = np.array([0]) 
        
        
        if "Time [s]" in session.df_stim.columns:
            event_uuid_grp.attrs["stim_start_end_time"] = np.array(session.df_stim["Time [s]"])
        else:
            event_uuid_grp.attrs["stim_start_end_time"] = np.array(session.df_stim["Time [m:s.ms]"])
            
        event_uuid_grp.attrs["stim_start_end_sw_time"] = np.array(session.df_stim["SW Time [s]"])  
        event_uuid_grp.attrs["stim_start_end_nidaq_time"] = np.array(session.df_stim["NIDAQ Time [s]"])  
    
    
        # get first imaging frame falling into stim, last imaging frame falling into stim
        
